In [261]:
import requests
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import urllib.parse
from csv import writer
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [267]:
stop_words = set(stopwords.words('english')) 
stemmer = WordNetLemmatizer()

In [34]:
# init it_industry word for search
it_industry = pd.read_csv("data/it_industry.csv", header=None)

In [212]:
# init indeed
base_url = "https://www.indeed.com/"
pages = 30 # scape 5 page each page include 10 job = 300job

In [224]:
def append_list_as_row(file_name, item):
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0 :
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(item)

In [227]:
def request_query(page):
    response = requests.get(page)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [249]:
def request_query_soup(keyword, page):
    query_search = f"{base_url}jobs?q={urllib.parse.quote(keyword)}&start={page}"
    return request_query(query_search)

In [271]:
def clean_data(document):
    document = re.sub(r'\W', ' ', str(document))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # remove hashtag #
    document = re.sub(r'#','',document) 
    
    # Converting to Lowercase
    document = document.lower()
    
    return document

In [272]:
def split_word(document):
    tokens = word_tokenize(clean_data(document))
    tokens = [i for i in tokens if not i in stop_words] # remove stop word
    tokens = [word for word in tokens if word.isalpha()] # remove alpha
    tokens = [i for i in tokens if not i.isnumeric()] # remove number
    tokens = [i for i in tokens if not ' ' in i] # remove space
    tokens = [stemmer.lemmatize(i) for i in tokens] # root
    
    document = ' '.join(tokens)
    return document

In [215]:
# loop each page
with tqdm(total=len(it_industry) * pages) as pbar:

    for keyword in it_industry[0]:
        for page in range(pages):
            page = page * 10
            soup = request_query_soup(keyword, page)

            link_list = []
            keyword_list = []

            query_div = soup.find_all("div", {"class": "jobsearch-SerpJobCard"})
            for div in query_div:
                link_list.append(f"https://www.indeed.com/viewjob?jk={div.attrs['data-jk']}&from=serp&vjs=3")
                keyword_list.append(keyword)

            df = pd.DataFrame({'job_type': keyword_list, 'link': link_list})
            df.to_csv('data/job_link.csv', mode='a', header=False, index=None)
            pbar.update(1)

100%|██████████| 540/540 [15:09<00:00,  1.68s/it]


In [259]:
# load job_link
job_link = pd.read_csv("data/job_link.csv", header=None)

In [280]:
# loof each link
header_list = []
company_list = []
desc_list = []
job_link_temp = job_link[1]

with tqdm(total=len(job_link_temp)) as pbar:
    for link in job_link_temp:
        soup = request(link)
        header = soup.find("h3", {"class": "jobsearch-JobInfoHeader-title"})
        header = header.getText() if header else ""
        company = soup.find("div", {"class": "jobsearch-InlineCompanyRating"})
        company = company.getText() if company else ""
        desc = soup.find("div", {"class": "jobsearch-jobDescriptionText"})
        desc = split_word(desc.getText()) if desc else ""

        df = pd.DataFrame({'job_title': [header], 'company': [company], 'desc': [desc]})
        df.to_csv('data/job_scrap.csv', mode='a', header=False, index=None)
        pbar.update(1)

100%|██████████| 5340/5340 [2:01:13<00:00,  1.36s/it]  


In [302]:
job_scrap = pd.read_csv("data/job_scrap.csv", header=None)

In [371]:
job_data = pd.concat([job_link, job_scrap], axis=1)
job_data.columns=['job_type', 'link', 'job_title', 'company', 'desc']
job_data.dropna(subset = ['job_title', 'desc'], inplace=True)
job_data.to_csv('data/job_data.csv', index=None)

PermissionError: [Errno 13] Permission denied: 'data/job_data.csv'